In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path 
import datetime
import logging

import numpy as np
import pandas as pd
import matplotlib as mpl 
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

from rowing.analysis import geodesy, files, splits, garmin, utils

In [3]:
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)

In [4]:
spread = "https://docs.google.com/spreadsheets/d/1tp0aS5jiKe5Ah-rfYjCKmxQSRo58lvIAGJGosKZMfX0/edit#gid=0"
path = "garmin-data"
credentials = "garmin-credentials.json"

In [5]:
today = pd.Timestamp.today().date()
yesterday = today - pd.Timedelta("1d")

In [6]:
self = garmin.GarminClient.from_credentials(
    credentials, local_cache=utils.json_cache, path=path
).login()

In [19]:
start = (
    pd.Timestamp.today().date() - pd.Timedelta("30D")
    - pd.offsets.MonthBegin(1)
)
activities = self.get_activities(start=start, end=pd.Timestamp.today())
positions = self.load_fits(activities.loc[
    activities['activityType.typeKey'] == 'rowing'
].activityId)

best_times = garmin.find_best_times(positions, activities)

for month, month_best in best_times.groupby(
    pd.Grouper(axis=1, level='time', freq='M'), axis=1
):
    if not month_best.empty:
        gspread_df = utils.format_gsheet(
            month_best.loc[
                month_best.notna().any(1), 
                month_best.notna().any(0), 
            ].sort_index(axis=1, ascending=False)
        )
        utils.to_gspread(
            gspread_df.T, spread, month.strftime("best times %Y-%m")
        )


  0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/97 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

In [17]:
best_times.groupby(
    pd.Grouper(axis=1, level='time', freq='M'), axis=1
).size()

time
2022-08-31    66
Freq: M, dtype: int64

In [7]:
activities = self.get_activities(period="2000D")
positions = self.load_fits(activities.loc[
    activities['activityType.typeKey'] == 'rowing'
].activityId)

  0%|          | 0/2001 [00:00<?, ?it/s]

/mnt/c/Users/matthew/Dropbox/PC/Documents/Source/rowing/rowing/analysis/garmin.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  activities['time'] = pd.to_datetime(activities.startTimeLocal)


  0%|          | 0/1738 [00:00<?, ?it/s]

In [124]:
weather_data, errors = utils.map_concurrent(
    self.get_weather,  
    dict(zip(activities.activityId, activities.activityId)), 
    singleton=True
)
weather = pd.DataFrame.from_dict(weather_data, orient='index')

  0%|          | 0/2758 [00:00<?, ?it/s]

In [40]:
stats, errors = utils.map_concurrent(
    self.get_stats, 
    list(pd.date_range("2018-01-01", "2022-09-13")), 
    singleton=True
)
year_stats = pd.DataFrame.from_records(stats).set_index("calendarDate").sort_index(ascending=False)

utils.to_gspread(
    year_stats, spread, "garmin stats"
)

  0%|          | 0/1717 [00:00<?, ?it/s]

In [8]:
best_times = garmin.find_best_times(positions, activities)

  0%|          | 0/1738 [00:00<?, ?it/s]

In [7]:
for month, month_best in best_times.groupby(
    pd.Grouper(axis=1, level='time', freq='M'), axis=1
):
    if not month_best.empty:
        gspread_df = utils.format_gsheet(
            month_best.loc[
                month_best.notna().any(1), 
                month_best.notna().any(0), 
            ].sort_index(axis=1, ascending=False)
        )
        utils.to_gspread(
            gspread_df.T, spread, month.strftime("best times %Y-%m")
        )


  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]